In [ ]:
import pandas as pd
import sqlite3

In [ ]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [ ]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()


In [ ]:
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()

OperationalError: table restaurants already exists

In [ ]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table'",
    conn
)

,name
0,restaurants


In [ ]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
orders_users = orders.merge(users, on="user_id", how="left")

final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

In [ ]:
orders.shape, orders_users.shape, final_df.shape

((10000, 6), (10000, 9), (10000, 12))

In [ ]:
final_df.head()
final_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [ ]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [ ]:
final_df = final_df.drop(columns=["restaurant_name_x"])
final_df = final_df.rename(columns={"restaurant_name_y": "restaurant_name"})

In [ ]:
final_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'name', 'city', 'membership', 'restaurant_name', 'cuisine', 'rating'],
      dtype='object')

In [ ]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

In [ ]:
df[df['membership'] == 'Gold'] \
  .groupby('city')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [ ]:
df.groupby('cuisine')['total_amount'] \
  .mean() \
  .sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [ ]:
user_spend = df.groupby('user_id')['total_amount'].sum()
user_spend[user_spend > 1000].count()

np.int64(2544)

In [ ]:
rating_bins = pd.cut(
    df['rating'],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0]
)

df.groupby(rating_bins)['total_amount'] \
  .sum()

/tmp/ipython-input-182218364.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(rating_bins)['total_amount'] \


,total_amount
rating,
"(3.0, 3.5]",1881754.57
"(3.5, 4.0]",1717494.41
"(4.0, 4.5]",1960326.26
"(4.5, 5.0]",2197030.75


In [ ]:
df[df['membership'] == 'Gold'] \
  .groupby('city')['total_amount'] \
  .mean() \
  .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [ ]:
df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
)

,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [ ]:
gold_orders = df[df['membership'] == 'Gold'].shape[0]
total_orders = df.shape[0]

round((gold_orders / total_orders) * 100)

50

In [ ]:
restaurant_stats = df.groupby('restaurant_name').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

restaurant_stats[restaurant_stats['total_orders'] < 20] \
    .sort_values('avg_order_value', ascending=False) \
    .head()

,avg_order_value,total_orders
restaurant_name,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16


In [ ]:
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered = df[df['restaurant_name'].isin(options)]

In [ ]:
filtered_stats = filtered.groupby('restaurant_name').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

filtered_stats[filtered_stats['total_orders'] < 20] \
    .sort_values('avg_order_value', ascending=False)

,avg_order_value,total_orders
restaurant_name,,


In [ ]:
filtered_stats.sort_values('avg_order_value', ascending=False)

,avg_order_value,total_orders
restaurant_name,,


In [ ]:
df.groupby(['membership', 'cuisine'])['total_amount'] \
  .sum() \
  .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

In [ ]:
df['quarter'] = df['order_date'].dt.quarter

df.groupby('quarter')['total_amount'] \
  .sum()

,total_amount
quarter,
1,2010626.64
2,1945348.72
3,2037385.10
4,2018263.66


In [ ]:
df = pd.read_csv("final_food_delivery_dataset.csv")
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

In [ ]:
gold_orders = df[df['membership'] == 'Gold'].shape[0]
gold_orders

4987

In [ ]:
hyderabad_revenue = round(
    df[df['city'] == 'Hyderabad']['total_amount'].sum()
)
hyderabad_revenue

1889367

In [ ]:
distinct_users = df['user_id'].nunique()
distinct_users

2883

In [ ]:
gold_avg_order_value = round(
    df[df['membership'] == 'Gold']['total_amount'].mean(),
    2
)
gold_avg_order_value

np.float64(797.15)

In [ ]:
high_rating_orders = df[df['rating'] >= 4.5].shape[0]
high_rating_orders

3374

In [ ]:
top_gold_city = (
    df[df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

orders_top_gold_city = df[
    (df['membership'] == 'Gold') &
    (df['city'] == top_gold_city)
].shape[0]

top_gold_city, orders_top_gold_city

('Chennai', 1337)